In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn
import torch

# No-Transaction-Band Network with Non-Linear Transaction Costs

This notebook demonstrates how to implement a no-transaction-band network in `pfhedge` with a non-linear transaction cost function. The transaction cost is proportional to the volatility of the stock raised to the power of 1.3 and the traded volume raised to the power of 0.75.

## Step 1: Import Required Libraries


In [2]:
import torch
from pfhedge.instruments import BrownianStock, EuropeanOption
from pfhedge.nn import Hedger



In [3]:
torch.manual_seed(42)

if not torch.cuda.is_available():
    raise RuntimeWarning(
        "CUDA is not available. "
        "If you're using Google Colab, you can enable GPUs as: "
        "https://colab.research.google.com/notebooks/gpu.ipynb"
    )


RuntimeWarning: CUDA is not available. If you're using Google Colab, you can enable GPUs as: https://colab.research.google.com/notebooks/gpu.ipynb

In [4]:

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Default device:", DEVICE)

Default device: cpu


In [5]:
# In each epoch, N_PATHS brownian motion time-series are generated.
N_PATHS = 50000
# How many times a model is updated in the experiment.
N_EPOCHS = 200

def to_numpy(tensor: torch.Tensor) -> np.array:
    return tensor.cpu().detach().numpy()

In [7]:
import torch.nn.functional as fn
from torch import Tensor
from torch.nn import Module
from pfhedge.nn import BlackScholes, Clamp, MultiLayerPerceptron


class NoTransactionBandNet(Module):
    def __init__(self, derivative):
        super().__init__()

        self.delta = BlackScholes(derivative)
        self.mlp = MultiLayerPerceptron(out_features=2)
        self.clamp = Clamp()

    def inputs(self):
        return self.delta.inputs() + ["prev_hedge"]

    def forward(self, input: Tensor) -> Tensor:
        prev_hedge = input[..., [-1]]

        delta = self.delta(input[..., :-1])
        width = self.mlp(input[..., :-1])

        min = delta - fn.leaky_relu(width[..., [0]])
        max = delta + fn.leaky_relu(width[..., [1]])

        return self.clamp(prev_hedge, min=min, max=max)

In [8]:
def non_linear_transaction_cost(prev_position, new_position, volatility=0.16, avg_volume=1e8):
    """
    Custom non-linear transaction cost function based on stock volatility.

    Args:
    - hedger: Hedger instance
    - prev_position: Portfolio positions at the previous step
    - new_position: Portfolio positions at the current step
    - market: Market instance
    - paths: Simulated paths of the stock price

    Returns:
    - cost: Computed transaction cost
    """
    # Calculate the change in position
    delta_position = new_position - prev_position

    # Calculate traded volume
    traded_volume = torch.abs(delta_position)

    # Apply the custom transaction cost formula
    cost = 42. * (volatility ** 1.3) * ((traded_volume/avg_volume) ** 0.75)
    return cost.sum()


In [37]:
import torch
from pfhedge.nn import Hedger
from pfhedge.instruments import EuropeanOption, BrownianStock

# Define the custom transaction cost function
class CustomTransactionCost:
    def __init__(self, cost_rate=0.01):
        """
        Initializes the transaction cost model.
        Args:
            cost_rate: Proportionality constant for transaction cost.
        """
        self.cost_rate = cost_rate

    def __call__(self, delta_change):
        """
        Computes transaction cost proportional to the 3/4 power of the delta change.
        Args:
            delta_change: The volume of stock traded (torch.Tensor).
        Returns:
            Transaction cost (torch.Tensor).
        """
        return self.cost_rate * torch.abs(delta_change) ** (3 / 4)



# Customize the training loop to incorporate transaction costs
def train_with_transaction_cost(hedger, option, transaction_cost_model):
    """
    Custom training loop that incorporates transaction costs into the hedger.
    Args:
        hedger: The hedger object to train.
        option: The option to hedge.
        transaction_cost_model: The transaction cost model to apply.
    """
    # Access the simulated spot prices from the underlying asset
    spot = option.underlier.spot  # Use the simulated spot buffer

    for path in spot.T:  # Iterate over each path
        # Compute inputs for the hedger model
        log_moneyness = torch.log(path / option.strike)
        time_to_maturity = torch.linspace(option.maturity, 0, len(path))

        # Compute deltas using the hedger model
        inputs = torch.stack([log_moneyness, time_to_maturity], dim=-1)
        deltas = hedger.model(inputs).squeeze()  # Predict deltas from the hedger model
        delta_changes = torch.diff(deltas, dim=0)
        
        # Apply the transaction cost function
        transaction_costs = transaction_cost_model(delta_changes)
        
        # Compute the option payoff
        payoff = option.payoff(path)
        
        # Adjust the profit and loss (P&L) for transaction costs
        pnl = payoff - transaction_costs.sum()
        
        # Update the hedger's parameters
        hedger.step(pnl)





In [41]:
# Define the underlying asset (Brownian motion stock process)
stock = BrownianStock(cost=5e-4)

# Define the derivative (European option) based on the stock
option = EuropeanOption(stock, strike=100, maturity=30/252)

# Define the transaction cost model
transaction_cost_model = CustomTransactionCost(cost_rate=0.01)


model = NoTransactionBandNet(derivative=option)

# Initialize the hedger without a specific model for now
hedger = Hedger(model=model, inputs=["log_moneyness", "time_to_maturity"])

/Users/Shared/pt_conda3/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [42]:
stock.simulate(n_paths=N_PATHS)
# Simulate paths of the underlying
option.simulate(n_paths=N_PATHS)

In [43]:

# Train the hedger with transaction costs
train_with_transaction_cost(hedger, option, transaction_cost_model)

# Evaluate the price and hedging strategy
hedge_price = hedger.price(option)
print(f"Hedger Price: {hedge_price}")


TypeError: BaseDerivative.payoff() takes 1 positional argument but 2 were given

In [45]:
option

EuropeanOption(
  strike=100, maturity=0.1190
  (underlier): BrownianStock(sigma=0.2000, cost=0.0005, dt=0.0040)
)